# 10장 케라스를 사용한 인공신경망 소개

* 인공신경망
    * 딥러닝의 핵심
    * 강력하며 확장성이 좋음

* 인공신경망의 응용: 대규모 머신러닝 문제 다루기에 적합
    * 구글 이미지: 수백만 개의 이미지 분류
    * 애플의 시리: 음성인식 서비스
    * 유튜브: 가장 좋은 비디오 추천
    * 딥마인드의 알파고: 스스로 기보를 익히면서 학습하는 바둑 프로그램

* 퍼셉트론
    * 가장 단순한 인공신경망 구조 중 하나
    * 프랑크 로젠블라트가 1957년에 발표.

## 주요 내용

* 퍼셉트론 소개
* 다층 퍼셉트론과 역전파
* 케라스(keras)를 이용한 다층 퍼셉트론 구현
* 텐서보드를 활용한 시각화
* 신경망 하이퍼파라미터 튜닝

## 퍼셉트론 소개

* TLU(threshold logic unit) 또는 LTU(linear threshold unit) 라 불리는 인공 뉴런 활용
* 입력/출력: 숫자
* 모든 입력은 가중치와 연결됨.

### TLU

* 입력값과 가중치의 곱한 값들의 합에 계단함수(step function) 적용.

\begin{align*}
z &= {\mathbf x}^T {\mathbf w} \\
  &= x_1 w_1 + x_2 w_2 + \cdots + x_n w_n \\
  & \\
h_\mathbf{w}(\mathbf x) &= step(z)
\end{align*}

* $n$: 특성 수

#### 예제

* 3개의 특성을 갖는 샘플 하나가 입력되면 가중치와 곱한 후 합을 계산
* 이후 계단함수를 통과시킨 결과를 출력
* 보통은 편향에 대한 가중치고 함께 고려. (잠시 뒤에 설명)

<img src="images/ch10/homl10-01.png" width="300"/>

### 계단함수

* 가장 많이 사용되는 계단함수: Heaviside 계단함수와 sign 함수

$$
\mathrm{heaviside}(z) = 
\begin{cases}
0 & \text{ if } z < 0\\
1 & \text{ if } z \ge 0
\end{cases}
\quad\qquad
\mathrm{sgn}(z) = 
\begin{cases}
-1 & \text{ if } z < 0\\
0 & \text{ if } z = 0\\
1 & \text{ if } z > 0
\end{cases}
$$

### TLU와 선형 이진분류

* 하나의 TLU를 간단한 이진분류기로 활용 가능

* 모든 입력값(특성)의 선형 조합을 계산한 후에 임곗값을 기준으로 양성/음성 분류

* 작동은 로지스틱 회귀 또는 선형 SVM 분류기와 비슷.

* TLU 모델 학습 = 최적의 가중치 $w_i$를 찾기

### 퍼셉트론 정의

* 하나의 층에 여러 개의 TLU로 구성됨.

* TLU 각각은 모든 입력과 연결됨.

#### 입력층

* 입력 뉴런으로 구성된 층

* 입력 뉴런: 입력을 받아 그대로 통과시켜 출력하는 뉴런

* 편향값 1을 항상 출력하는 편향 뉴런과 여러 개의 입력 뉴런이 함께 사용됨.

#### 예제

* 입력 두 개와 출력 세 개로 구성된 퍼셉트론

<img src="images/ch10/homl10-02.png" width="400"/>

#### 완전연결 층(밀집 층)

* 층에 속한 각각의 뉴런이 이전 층의 모든 뉴런과 연결되어 있을 때를 가리킴.

* 예제: 퍼셉트론의 출력층

### 인공뉴런 층

* 인공뉴런 층은 여러 개의 뉴런으로 구성됨.
* 여러 샘플에 대한 출력을 행렬 연산을 이용하여 효율적으로 계산할 수 있음.

### 완전연결 층의 출력 계산

* 여러 개의 완전연렬 층으로 구성된 인공신경망 일반 적인 형태

<img src="images/ch10/homl10-04.png" width="350"/>

<그림 출처: [netclipart](https://www.netclipart.com/down/ihwJhJh_angle-symmetry-area-neural-networks-transparent-backgrounds/)>

* 이 중에서 아래 빨간색으로 이루어진 부분에 대한 계산은 어떻게?

<img src="images/ch10/homl10-05.png" width="350"/>

* 다음과 같이 이루어짐.

<img src="images/ch10/homl10-03.png" width="400"/>

<그림 출처: [medium](https://medium.com/@jayeshbahire/the-artificial-neural-networks-handbook-part-4-d2087d1f583e)>

* 하나의 층에서 이루어지는 입력과 출력을 행렬 수식으로 표현 가능

<img src="images/ch10/homl10-06.png" width="350"/>

\begin{align*}
h_{\mathbf{w},\mathbf{b}}(\mathbf X) = \phi({\mathbf X}{\mathbf W} + {\mathbf b})
\end{align*}

\begin{align*}
h_{\mathbf{w},\mathbf{b}}(\mathbf X) = \phi({\mathbf X}{\mathbf W} + {\mathbf b})
\end{align*}

* $\mathbf X$: 입력 샘플들의 특성행렬. 
    * 각 행은 샘플을 가리킴
    * 각 열은 특성을 가리킴

* $\mathbf W$: 편향 뉴런을 제외한 모든 뉴런에 대한 가중치.
    * 각 행은 하나의 입력과 연관됨
    * 각 열은 하나의 출력과 연관됨.

* $\mathbf b$: 편향벡터
    * 편향 뉴런과 연결된 각각의 출력에 대한 편향값으로 구성된 벡터

* $\phi$: 활성화 함수(activation function)
    * 각 인공뉴런이 TLU인 경우, 계단함수가 사용됨.

### 퍼셉트론 학습 알고리즘

* 오차가 감소되는 방향으로 가중치를 조절하는 방식으로 뉴런 사이의 관계를 강화시킴.

* 하나의 샘플이 입력될 때 마다 예측한 후에 오차를 계산하여 오차가 줄어드는 방향으로 가중치 조절.

$$
w_{i, j}^{(\text{next step})} = 
w_{i, j} + \eta\, (y_j - \hat y_j)\, x_j
$$

$$
w_{i, j}^{(\text{next step})} = 
w_{i, j} + \eta\, (y_j - \hat y_j)\, x_j
$$

* $w_{i, j}$: 가중치 행렬 $W$의 $i$행 $j$열의 값.

* $x_{i}$: 입력 샘플의 $i$번째 속성

* $\hat y_{j}$: $j$번째 출력값

* $y_{j}$: $j$번째 출력값에 대한 타깃

* $\eta$: 학습률

### 퍼셉트론과 선형성

* 각 출력 뉴런의 결정경계가 선형

* 따라서 퍼셉트론도 복잡한 패턴 학습 못함. 하지만 ...

* 퍼셉트론 수렴 이론: 선형적으로 구분될 수 있는 모델은 언제나 학습 가능

### 사이킷런의 퍼셉트론 모델

* Perceptron 클래스 활용

#### 예제

* 붓꽃 데이터셋 분류

```python
import numpy as np
from sklearn.datasets import load_iris
from sklearn.linear_model import Perceptron

iris = load_iris()
X = iris.data[:, (2, 3)]  # petal length, petal width
y = (iris.target == 0).astype(np.int)

per_clf = Perceptron(max_iter=1000, tol=1e-3, random_state=42)
per_clf.fit(X, y)
```

<img src="images/ch10/homl10-07.png" width="600"/>

### 퍼셉트론의 특징

* 아래 옵션을 사용할 경우 SGDClassifier 와 동일하게 작동

    loss="perceptron", learning_rate="constant", eta0=1, penalty=None

* 클래스 확률 지원 없음. 따라서 로지스틱 회귀가 보다 선호됨.

* 퍼셉트론은 매우 단순한 경우만 해결 가능. 

* 하지만 퍼셉트론을 여러 개 쌓아올리면 꽤 강력한 인공신경망 구성함.

## 다층 퍼셉트론(MLP)과 역전파

* 다층 퍼셉트론(multilayer perceptron, MLP): 퍼셉트론을 여러 개 쌓아올린 인공신경망

* 구성은 다음과 같음.

    * 한 개의 입력층
    * 여러 개의 은닉층
    * 한 개의 출력층

<img src="images/ch10/homl10-08.png" width="400"/>

### 심층신경망(DNN)

* 여러 개의 은닉층을 쌓아올린 인공신경망

### 역전파 훈련 알고리즘

* 다층 퍼셉트론은 층이 많을 수록 훈련시키는 과정이 점점 더 어려워짐.

* 1986년에 소개된 역전파(backpropagation) 훈련 알고리즘이 발표된 이후로 실용성 갖춤.

* 1단계(정방향): 각 훈련 샘플에 대해 먼저 예측을 만든 후 오차 측정

* 2단계(역방향): 역방향으로 각 층을 거치면서 각 연결이 오차에 기여한 정도 측정

* 3단계: 오차가 감소하도록 모든 가중치 조정

### MLP 특징

* 랜덤하게 설정함.

    * 그렇지 않으면 층의 모든 뉴련이 동일하게 움직임.

* 활성환 함수 교체: 보통 계단함수 대신에 다른 함수 사용.
    * 로지스틱(시그모이드)
    * 하이퍼볼릭 탄젠트 함수(쌍곡 탄젠트 함수)
    * ReLU 함수

<img src="images/ch10/homl10-09.png" width="400"/>

### 활성화 함수 필요성

* 선형성을 벗어나기 위해.
    * 선형 변환을 여러 개 연결 해도 선형 변환에 머무름.
    * 따라서 복잡한 문제 해결 불가능.

* 비선형 활성화 함수를 충분히 많은 층에서 사용하면 매우 강력한 모델 학습 가능

### 회귀를 위한 MLP

#### 출력 뉴런 수

* 예측해야 하는 값의 수에 따라 출력 뉴런 설정
    * 주택 가격 예측: 출력 뉴런 1개
    * 다변량 회귀(동시에 여러값 예측하기): 출력 차원마다 출력 뉴런 1개
        * 물체의 중심 위치: 좌표 2개 필요, 따라서 출력 뉴런 2개 필요.

#### 활성화 함수 지정

* 출력값에 특별한 제한이 없다면 활성화 함수 사용하지 않음.

* 출력이 양수인 경우
    * ReLU 또는 softplus 사용 가능

<img src="images/ch10/homl10-10.png" width="350"/>

<그림 출처: [위키피디아](https://en.wikipedia.org/wiki/Rectifier_(neural_networks))>

* 출력이 특정 범위 안에 포함되어야 할 경우
    * 로지스틱 함수 또는 하이퍼볼릭 탄젠트 함수와 적절한 스케일 조정 활용

#### 손실함수

* 일반적으로 평균제곱오차(MSE) 활용

* 이상치가 많을 경우: 평균 절댓값 오차 사용 가능

* 후버(Huber) 손실 사용 가능
    * MSE와 평균 절댓값 오차의 조합

#### 회귀 MLP의 전형적인 구조

| 하이퍼파라미터 | 일반적으로 사용되는 값 |
| :--- | :---|
| 입력뉴런 수| 샘플의 특성마다 하나 |
| 은닉층 수 | 문제에 따라 다름. 보통 1-5개 |
| 은닉층의 뉴런 수 | 문제에 따라 다름. 보통 10-100개 |
| 출력뉴런 수 | 예측 차원마다 하나 |
| 은닉층의 활성화 함수 | ReLU 또는 SELU(11장 참조) |
| 출력층의 활성화 함수 | 보통 없음. 상황에 따라 ReLU/softplus 또는 logistic/tanh 사용 |
| 손실함수 | MSE 또는 MAE/Huber(이상치 많은 경우) |

### 분류를 위한 다층 퍼셉트론

## 케라스(keras)를 이용한 MLP 구현

## 텐서보드를 활용한 시각화

## 신경망 하이퍼파라미터 튜닝